**RNN with variable length sequence**

In [1]:
import tensorflow as tf
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/RNN_variableLength_Sequence')
import pickle
# from prepare_data import parse_seq
from prepare_data2 import parse_seq

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **RE-IMPLEMENTATION**

In [ ]:
!python3 prepare_data.py shakespeare_input.txt sk

2020-06-01 20:58:12.631217: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [ ]:
# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("sk.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))
# data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("sk_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'C': 1, '!': 2, '.': 3, ' ': 4, 'G': 5, 'f': 6, 'i': 7, 'X': 8, 'I': 9, 'N': 10, 'c': 11, ']': 12, 'Z': 13, '[': 14, 'B': 15, 'L': 16, 'g': 17, 'K': 18, ':': 19, '&': 20, ';': 21, 'd': 22, 'O': 23, ',': 24, 'Y': 25, 'Q': 26, 'z': 27, 'h': 28, 'q': 29, 'l': 30, "'": 31, 's': 32, 'A': 33, 'V': 34, '-': 35, 'n': 36, 'v': 37, 'u': 38, 'r': 39, 'j': 40, 't': 41, 'a': 42, 'y': 43, 'M': 44, 'W': 45, 'H': 46, 'T': 47, 'e': 48, 'p': 49, '3': 50, 'k': 51, 'S': 52, '?': 53, 'E': 54, 'R': 55, '\n': 56, 'b': 57, 'U': 58, 'x': 59, 'o': 60, 'w': 61, 'D': 62, 'P': 63, 'J': 64, '$': 65, 'm': 66, 'F': 67, '<S>': 0}
68


In [ ]:
for ind, thing in enumerate(data):
    inds = thing.numpy()
    to_chars = "".join([ind_to_ch[ind] for ind in inds])
    # print(ind)
    print(to_chars)
    #print()
    break

<S>First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, yo


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = data.map(split_input_target)
dataset

In [ ]:
# for input_example, target_example in  dataset.take(1):
#   print ('Input data: ', repr(''.join(idx2chr[input_example.numpy()])))
#   print ('Target data:', repr(''.join(ind_to_ch[target_example.numpy()])))

# to_chars = "".join([ind_to_ch[ind] for ind in inds])

for input_example, target_example in  dataset.take(1):
  input_example = input_example.numpy().flatten()
  target_example = target_example.numpy().flatten()
  print ('Input data :',  repr("".join([ind_to_ch[ind] for ind in input_example])))
  print ('Target data:', repr("".join([ind_to_ch[ind] for ind in target_example])))

Input data : '<S>First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, y'
Target data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, yo'


In [ ]:
dataset = dataset.shuffle(22981).batch(128, drop_remainder= True)
embedding_dim = 512
rnn_units = 1024
batch_size = 128

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential(name= 'GRU')
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.GRU(rnn_units, return_sequences= True, stateful= True, recurrent_initializer= 'glorot_uniform'))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model


model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
model.summary()

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 512)          34816     
_________________________________________________________________
gru (GRU)                    (128, None, 1024)         4724736   
_________________________________________________________________
dense (Dense)                (128, None, 68)           69700     
Total params: 4,829,252
Trainable params: 4,829,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_ind = tf.random.categorical(example_batch_predictions[0], num_samples= 1)
sampled_ind = tf.squeeze(sampled_ind, axis= -1).numpy()

# sampled_ind
input_example_batch1 = input_example_batch[0].numpy()
# input_example_batch1
print("Input: \n", repr("".join([ind_to_ch[ind] for ind in input_example_batch1])))
print()
print("Next Char Predictions: \n", repr("".join([ind_to_ch[ind] for ind in sampled_ind])))

(128, 199, 68) # (batch_size, sequence_length, vocab_size)
Input: 
 '<S>e the ruin of your love\nAnd be my vantage to exclaim on you.\n\nBASSANIO:\nMadam, you have bereft me of all words,\nOnly my blood speaks to you in my veins;\nAnd there is such confusion in my powers,\nAs '

Next Char Predictions: 
 "iTJW?EabZPVQaUwqas\nb3LbGEQY-ffPZ.!\n\n:B'CNIK$Tpi?LJx.NSUeYCYBzU&wYKC\nnv?uloD?;lTtvUpQmDR33LbgkaKtR'HLhJd$Q&c\nv;Btj?'lVKaWF?F]$'znzTp3gjBcHu?JPccjXNdS!TOScyvl;;DhvpdnZ<S> :g'JGKKRJ\nwD'MG-A.n,xXW3FWsidx?K"


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits= True)


example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Mean Loss: ", example_batch_loss.numpy().mean())

Mean Loss:  4.220621


In [ ]:
model.compile(optimizer= 'Adam', loss= loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs= 10, callbacks=[checkpoint_callback])

Epoch 1/10
179/179 [==============================] - 90s 501ms/step - loss: 2.5429
Epoch 2/10
179/179 [==============================] - 90s 500ms/step - loss: 1.8072
Epoch 3/10
179/179 [==============================] - 90s 502ms/step - loss: 1.5486
Epoch 4/10
179/179 [==============================] - 90s 500ms/step - loss: 1.4256
Epoch 5/10
179/179 [==============================] - 90s 502ms/step - loss: 1.3580
Epoch 6/10
179/179 [==============================] - 90s 502ms/step - loss: 1.3147
Epoch 7/10
179/179 [==============================] - 90s 501ms/step - loss: 1.2827
Epoch 8/10
179/179 [==============================] - 90s 503ms/step - loss: 1.2570
Epoch 9/10
179/179 [==============================] - 90s 500ms/step - loss: 1.2348
Epoch 10/10
179/179 [==============================] - 90s 501ms/step - loss: 1.2156


In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size= 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            34816     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           4724736   
_________________________________________________________________
dense_1 (Dense)              (1, None, 68)             69700     
Total params: 4,829,252
Trainable params: 4,829,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      # text_generated.append(idx2char[predicted_id])
      text_generated.append(ind_to_ch[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, "JULIET: "))

JULIET: 

HELENA:
Of nothing, it's good, but on their neighbour,
And survy your thoughts to more names:--but, to cry so
That I at still never leave them as I
Hang it me. Good Master Sly! o heavens!
His creuts makes lords and unspossily.

APEMANTUS:
Pray, I'll be glad of thine all: where's aught?

WESTMIRET:
You shall not, to receive
Your friend wild conjoiced by this comfort,
Whose wing, when his cruelity and hervelles
like refuse, till he fall of heaving as married men's
open, wickedny of the cold fortunes that of
such a kingdom? stains so weak a maid most true, take
I as many fearfully unto this sport,
And I revoice good month, I'll do thy fit:
I am not it against some of Cordal,
Thou shalt enluce his troop. Neither! Flattering sleep,
As to 'twere: and therefore anster such service.

MISTRESS PAGE:
Has promised you as he rich: if you after day.

CLEON:

SPEED:
Why, follow me, and thou ways us, for she cried,
And mine have none farewell.

Fipst
Lords, then, if you know thou liegest ma

### **VARIABLE LENGTH SEQUENCES**

In [3]:
!python3 prepare_data2.py shakespeare_input.txt sp \\n\\n+ -m 100

2020-06-01 20:49:52.875101: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 31022 sequences...
Longest sequence is 3094 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 100 characters...
18677 sequences remaining.
Longest remaining sequence has length 99.
Removing length-0 sequences...
18677 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...

In [166]:
# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("sp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
# data = data.map(lambda x: parse_seq(x, 200))
data = data.map(lambda x: parse_seq(x))
dataset = data.padded_batch(batch_size= 128, padded_shapes= 100, drop_remainder= True)

# a map from characters to indices
vocab = pickle.load(open("sp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{';': 3, ':': 4, 'E': 5, '-': 6, 's': 7, 'A': 8, 'v': 9, 'j': 10, 'C': 11, 'V': 12, 'r': 13, 'R': 14, 'B': 15, 'P': 16, 'b': 17, 'e': 18, '&': 19, 'u': 20, 'S': 21, 'n': 22, 'g': 23, '\n': 24, 'L': 25, 'h': 26, '!': 27, 'H': 28, 'w': 29, 'p': 30, '[': 31, 'Q': 32, ' ': 33, '.': 34, 'T': 35, '$': 36, ',': 37, 'O': 38, 'I': 39, 'W': 40, 'q': 41, 'N': 42, 'F': 43, ']': 44, 't': 45, 'm': 46, 'X': 47, 'y': 48, 'Y': 49, "'": 50, 'x': 51, 'Z': 52, 'z': 53, 'D': 54, 'J': 55, 'a': 56, 'd': 57, 'M': 58, '?': 59, 'G': 60, 'f': 61, 'c': 62, 'o': 63, 'K': 64, 'k': 65, '3': 66, 'U': 67, 'l': 68, 'i': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}
70


In [167]:
for ind, thing in enumerate(dataset):
    inds = thing[0].numpy()
    thing = thing.numpy()
    # print(thing)
    # print(tf.shape(thing))
    to_chars = "".join([ind_to_ch[ind] for ind in inds])
    # print(ind)
    print(to_chars)
    #print()
    break

<S>First Citizen:
Before we proceed any further, hear me speak.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>


In [168]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = dataset.map(split_input_target)
dataset

<MapDataset shapes: ((127, 100), (127, 100)), types: (tf.int32, tf.int32)>

In [169]:
for input_example, target_example in  dataset.take(1):
  input_example = input_example.numpy().flatten()
  print(input_example)
  target_example = target_example.numpy().flatten()
  print ('Input data :',  repr("".join([ind_to_ch[ind] for ind in input_example])))
  print ('Target data:', repr("".join([ind_to_ch[ind] for ind in target_example])))

[ 1 43 69 ...  0  0  0]
Input data : "<S>First Citizen:\nBefore we proceed any further, hear me speak.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><S>All:\nSpeak, speak.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><S>First Citizen:\nYou are all resolved rather to die than to famish?</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><S>All:\nResolved. resolved.</S><PAD><PA

In [ ]:
# dataset = dataset.shuffle(200).padded_batch(128, padded_shapes= 200, drop_remainder= True)
embedding_dim = 256
rnn_units = 512
batch_size = 128

In [171]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
  embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None], mask_zero= True)
  rnn = tf.keras.layers.GRU(rnn_units, return_sequences= True, stateful= True, recurrent_initializer= 'glorot_uniform')
  # rnn = tf.keras.layers.SimpleRNN(rnn_units, return_sequences= True, stateful= True, recurrent_initializer= 'glorot_uniform')
  dense = tf.keras.layers.Dense(vocab_size)
  layer_list = [embedding, rnn, dense]

  model = tf.keras.Sequential(layer_list, name= 'SimpleRNN')
  return model


model = build_model(vocab_size, embedding_dim, rnn_units, batch_size-1)
model.summary()

Model: "SimpleRNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (127, None, 256)          17920     
_________________________________________________________________
gru_1 (GRU)                  (127, None, 512)          1182720   
_________________________________________________________________
dense_15 (Dense)             (127, None, 70)           35910     
Total params: 1,236,550
Trainable params: 1,236,550
Non-trainable params: 0
_________________________________________________________________


In [172]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_ind = tf.random.categorical(example_batch_predictions[0], num_samples= 1)
sampled_ind = tf.squeeze(sampled_ind, axis= -1).numpy()

# sampled_ind
input_example_batch1 = input_example_batch[0].numpy()
# input_example_batch1
print("Input: \n", repr("".join([ind_to_ch[ind] for ind in input_example_batch1])))
print()
print("Next Char Predictions: \n", repr("".join([ind_to_ch[ind] for ind in sampled_ind])))

(127, 100, 70) # (batch_size, sequence_length, vocab_size)
Input: 
 '<S>First Citizen:\nBefore we proceed any further, hear me speak.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

Next Char Predictions: 
 ',]GMI3<S>R]h3&wKtGTl$PV&JF<S>TWvrTVByQPIJNeUuD,yT]On-thm.DCSiz-dW&<PAD>BGJOTszZPwiUAK<S>fEYQTZ<PAD>qZbmK[zEuYKM,h['


In [173]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits= True)


example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Mean Loss: ", example_batch_loss.numpy().mean())

Mean Loss:  4.249303


In [ ]:
# n_h = 512
# w_xh = tf.Variable(tf.initializers.glorot_uniform()([vocab_size, n_h]))
# w_hh = tf.Variable(tf.initializers.glorot_uniform()([n_h, n_h]))
# b_h = tf.Variable(tf.zeros([n_h]))

# w_ho = tf.Variable(tf.initializers.glorot_uniform()([n_h, vocab_size]))
# b_o = tf.Variable(tf.zeros([vocab_size]))

# all_vars = [w_xh, w_hh, b_h, w_ho, b_o]

In [194]:
# # model.compile(optimizer= 'Adam', loss= loss)
# steps = 20*3500 // 128
# opt = tf.optimizers.Adam()
# loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# @tf.function
# def run_rnn_on_seq(seq_batch):
#     with tf.GradientTape() as tape:

#         state = tf.zeros([tf.shape(seq_batch)[0], n_h])
#         total_loss = tf.constant(0.)
#         # for time_step in tf.range(tf.shape(seq_batch)[1] - 1):
#         for time_step in tf.range(tf.shape(seq_batch)[1] - 1):
            
#             # print("time_step: ", seq_batch)
#             # inp_mask_out = tf.math.count_nonzero(input= seq_batch)

#             # inp_here = tf.one_hot(seq_batch[:, time_step], vocab_size)
#             # print("mask out: ", inp_mask_out)
#             print(seq_batch)


#             inp_mask_out = tf.math.count_nonzero(input= seq_batch, axis= 1, keepdims= True)
#             print(inp_mask_out)
#             mask = tf.sequence_mask(inp_mask_out[:-1], dtype= tf.dtypes.float32)
#             print(mask)

#             # state = tf.nn.tanh(tf.matmul(mask, w_xh) + tf.matmul(state, w_hh) + b_h)
#             # logits = tf.matmul(state, w_ho) + b_o

#             out = model(mask)
#             print(out)

#             loss_here = loss(seq_batch[1:], out)
#             total_loss += loss_here
#             # print(total_loss)
            
#         # total_loss /= tf.cast(tf.shape(seq_batch)[1] - 1, tf.float32)
#         total_loss = total_loss/99.0
#         # print(total_loss)
#     all_vars = model.trainable_variables
#     grads = tape.gradient(total_loss, all_vars)
    
#     # this is gradient clipping
#     glob_norm = tf.linalg.global_norm(grads)
#     grads = [g/glob_norm for g in grads]

#     model.reset_states()
    
#     opt.apply_gradients(zip(grads, all_vars))

#     return total_loss


model.compile(optimizer= 'Adam', loss= loss)

steps = 20*3500 // 128
opt = tf.optimizers.Adam()

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs= 10, callbacks=[checkpoint_callback])


# @tf.function
# def run_rnn_on_seq(seq_batch):
#   inp_seq = seq_batch[:-1]
#   # print(inp_seq)
#   target_batch = seq_batch[1:]
#   out = model(inp_seq)
#   seq_loss = tf.keras.losses.sparse_categorical_crossentropy(target_batch, out, from_logits= True)
#   # seq_loss = loss(target_batch, out)
        
#   non_zero = tf.TensorArray(tf.int64,size= 127)
#   for seq in tf.range(127):
#       count = tf.math.count_nonzero(input_example_batch[seq])-1
#       non_zero.write(seq,count).mark_used()
#   non_zero=non_zero.stack()
#   mask = tf.sequence_mask(non_zero,maxlen= 100,dtype=tf.float32)
#   loss = tf.math.multiply(mask,seq_loss)
#   loss = tf.reduce_sum(loss,axis=1)
#   loss = loss/tf.cast(non_zero,tf.float32)
#   batch_loss = tf.reduce_mean(loss)

#   return batch_loss

# def run_rnn_on_seq(seq_batch):
# # for step, seqs in enumerate(dataset):
#   epochs = 10
#   for epoch in range(epochs):
#     with tf.GradientTape() as tape:
#       inp_seq = seq_batch[:-1]
#       # print(inp_seq)
#       target_batch = seq_batch[1:]
#       out = model(inp_seq)
#       seq_loss = tf.keras.losses.sparse_categorical_crossentropy(target_batch, out, from_logits= True)
#       # seq_loss = loss(target_batch, out)
      
#       non_zero = tf.TensorArray(tf.int64,size= 127)
#       for seq in tf.range(127):
#         count = tf.math.count_nonzero(input_example_batch[seq])-1
#         non_zero.write(seq,count).mark_used()
#       non_zero=non_zero.stack()
#       mask = tf.sequence_mask(non_zero,maxlen= 100,dtype=tf.float32)
#       loss = tf.math.multiply(mask,seq_loss)
#       loss = tf.reduce_sum(loss,axis=1)
#       loss = loss/tf.cast(non_zero,tf.float32)
#       batch_loss = tf.reduce_mean(loss)

#       # xent = run_rnn_on_seq(seqs)

#       all_vars = model.trainable_variables
#       grads = tape.gradient(batch_loss, all_vars)
      
#     model.reset_states()
    
#     opt.apply_gradients(zip(grads, all_vars))
#     # if not step % 200:
#     #   print("Step: {} Loss: {}".format(step, xent_avg))
#     #   print()

#     # if step > steps:
#     #     break

#   return batch_loss


# history = model.fit(dataset, epochs= 10, callbacks=[checkpoint_callback])

Epoch 1/10
9338/9338 [==============================] - 151s 16ms/step - loss: 1.3688
Epoch 2/10
9338/9338 [==============================] - 151s 16ms/step - loss: 1.3601
Epoch 3/10
9338/9338 [==============================] - 149s 16ms/step - loss: 1.3519
Epoch 4/10
9338/9338 [==============================] - 149s 16ms/step - loss: 1.3434
Epoch 5/10
9338/9338 [==============================] - 149s 16ms/step - loss: 1.3355
Epoch 6/10
9338/9338 [==============================] - 148s 16ms/step - loss: 1.3308
Epoch 7/10
9338/9338 [==============================] - 148s 16ms/step - loss: 1.3310
Epoch 8/10
9338/9338 [==============================] - 149s 16ms/step - loss: 1.3375
Epoch 9/10
9338/9338 [==============================] - 148s 16ms/step - loss: 1.3506
Epoch 10/10
9338/9338 [==============================] - 148s 16ms/step - loss: 1.3508


In [123]:
for step, seqs in enumerate(dataset):
# for step, seqs in dataset.take(1):
    # if step == 0:
    xent_avg = run_rnn_on_seq(seqs)

    if not step % 200:
        print("Step: {} Loss: {}".format(step, xent_avg))
        print()
        

    if step > steps:
        break

Step: 0 Loss: 2.8698692321777344



In [189]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size= 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "SimpleRNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (1, None, 256)            17920     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 512)            1182720   
_________________________________________________________________
dense_18 (Dense)             (1, None, 70)             35910     
Total params: 1,236,550
Trainable params: 1,236,550
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from prepare_data import chs_to_inds
char2idx = {u:i for i, u in enumerate(vocab)}

def generate_text(model, start_string):
# def generate_text(model):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      # text_generated.append(idx2char[predicted_id])
      text_generated.append(ind_to_ch[predicted_id])

  return (start_string + ''.join(text_generated))

In [193]:
print(generate_text(model, "JULIET: "))

UnknownError: ignored

In [195]:
dataset = data.padded_batch(batch_size= 2, padded_shapes= 100, drop_remainder= True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = dataset.map(split_input_target)
print(dataset)

for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_ind = tf.random.categorical(example_batch_predictions[0], num_samples= 1)
sampled_ind = tf.squeeze(sampled_ind, axis= -1).numpy()

# sampled_ind
input_example_batch1 = input_example_batch.numpy()
# input_example_batch1
print("Input: \n", repr("".join([ind_to_ch[ind] for ind in input_example_batch1])))
print()
print("Next Sequence Predictions: \n", repr("".join([ind_to_ch[ind] for ind in sampled_ind])))

<MapDataset shapes: ((1, 100), (1, 100)), types: (tf.int32, tf.int32)>
(1, 100, 70) # (batch_size, sequence_length, vocab_size)


TypeError: ignored

In [ ]:
# import numpy as np

# def sample(n_steps):
#     state = tf.zeros([1, n_h])
#     gen = [0]

#     for step in range(n_steps):
#         state = tf.nn.tanh(tf.matmul(tf.one_hot(gen[-1:], depth=vocab_size), w_xh) + tf.matmul(state, w_hh) + b_h)
#         probs = tf.nn.softmax(tf.matmul(state, w_ho) + b_o).numpy()[0]
#         #gen.append(np.argmax(probs))  # use argmax instead of choice if you want

#         gen.append(np.random.choice(vocab_size, p=out))
#         if step==1:
#           print(tf.shape(gen))
#     return "".join([ind_to_ch[ind] for ind in gen])
        
# agg = sample(2000)

In [ ]:
# print(agg)